In [83]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pickle

In [86]:
sessions_file_path = "saved_sessions.pkl"

In [85]:
# Read Pickle data back into a dictionary
with open(sessions_file_path, "rb") as pickle_file:
    sessions = pickle.load(pickle_file)

print(sessions)

{'NPC_Manager': [{'role': 'user', 'content': ChatCompletionMessage(content='```\nPlease provide the output in this format. I will use it to create the two generic agents and their conflict.\n\nHere is my attempt at generating the background information:\n\n```xml\n<background class="Jett">\nJett grew up in a small town with a strong sense of community. He was always fascinated by technology and tinkering, spending hours in his father\'s garage workshop learning how to fix things. As he got older, Jett realized that his passion for tech could be a career, and he decided to pursue a degree in computer science.\n</background>\n\n<background class="Knox">\nKnox comes from a big city where individuality is celebrated. He was always drawn to the arts, spending most of his free time drawing and writing poetry. Knox never felt like he quite fit in with his peers, but he loved expressing himself through art. As he got older, Knox realized that his creative passion could be a career, and he deci

In [61]:
def get_session(chat_id):
    if chat_id not in sessions:
        sessions[chat_id] = []
    return sessions[chat_id]  

In [62]:
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
model = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

In [63]:
def send_system_preset(sessionID, preset):
    session = get_session(sessionID)
    completion = client.chat.completions.create(
        model=model,
        messages= session + [
            {"role": "system", "content": preset}
        ],
        temperature=0.7,
    )
    session.append({"role": "system", "content": completion.choices[0].message})
    return completion.choices[0].message.content

In [64]:
def send_chat(sessionID, chat):
    session = get_session(sessionID)
    completion = client.chat.completions.create(
        model=model,
        messages= session + [
            {"role": "user", "content": chat}
        ],
        temperature=0.7,
    )
    session.append({"role": "user", "content": completion.choices[0].message})
    return completion.choices[0].message.content

In [65]:
with open('NPCmanager_prompt.txt', 'r') as f:
    NPCmanager_prompt = f.read()

init_scenario = send_chat('NPC_Manager', NPCmanager_prompt)

In [66]:
Bs_data = BeautifulSoup(init_scenario, "html.parser")

In [67]:
background_jett = Bs_data.find('background', class_="Jett")
background_knox = Bs_data.find('background', class_="Knox")
met = Bs_data.find('met')
bond = Bs_data.find('bond')
pastime = Bs_data.find('pastime')
conflict = Bs_data.find('conflict')
pov_jett = Bs_data.find('pov', class_="Jett")
pov_knox = Bs_data.find('pov', class_="Knox")

In [68]:
jett_init_text = "You are role playing as a character called Jett who is recently having a conflict with her close friend Knox. Player is trying to resolve the conflict and get you and Knox to talk to each other again, but you and Knox are bitter about the situation and it is difficult to talk to about it at first."
knox_init_text = "You are role playing as a character called Knox who is recently having a conflict with his close friend Jett. Player is trying to resolve the conflict and get you and Jett to talk to each other again, but you and Jett are bitter about the situation and it is difficult to talk to about it at first."

In [69]:
jett_full_prompt = f"{jett_init_text}\n\n=== Background ===\n\n{background_jett}\n{met}\n{bond}\n{pastime}\n\n=== Conflict ===\n\n{conflict}\n\n=== POV ===\n\n{pov_jett}"
knox_full_prompt = f"{knox_init_text}\n\n=== Background ===\n\n{background_knox}\n{met}\n{bond}\n{pastime}\n\n=== Conflict ===\n\n{conflict}\n\n=== POV ===\n\n{pov_knox}"

In [70]:
jett_full_prompt

'You are role playing as a character called Jett who is recently having a conflict with her close friend Knox. Player is trying to resolve the conflict and get you and Knox to talk to each other again, but you and Knox are bitter about the situation and it is difficult to talk to about it at first.\n\n=== Background ===\n\n<background class="Jett">\nJett grew up in a small town with a strong sense of community. He was always fascinated by technology and tinkering, spending hours in his father\'s garage workshop learning how to fix things. As he got older, Jett realized that his passion for tech could be a career, and he decided to pursue a degree in computer science.\n</background>\n<met>\nJett and Knox met during their freshman year of college at a campus coffee shop. Jett was working on a project for class and needed help debugging some code. Knox, being an art major, didn\'t know much about coding, but he offered to help Jett brainstorm solutions. They spent hours talking about thei

In [71]:
knox_full_prompt

'You are role playing as a character called Knox who is recently having a conflict with his close friend Jett. Player is trying to resolve the conflict and get you and Jett to talk to each other again, but you and Jett are bitter about the situation and it is difficult to talk to about it at first.\n\n=== Background ===\n\n<background class="Knox">\nKnox comes from a big city where individuality is celebrated. He was always drawn to the arts, spending most of his free time drawing and writing poetry. Knox never felt like he quite fit in with his peers, but he loved expressing himself through art. As he got older, Knox realized that his creative passion could be a career, and he decided to pursue a degree in fine arts.\n</background>\n<met>\nJett and Knox met during their freshman year of college at a campus coffee shop. Jett was working on a project for class and needed help debugging some code. Knox, being an art major, didn\'t know much about coding, but he offered to help Jett brain

In [84]:
with open(sessions_file_path, "wb") as pickle_file:
    pickle.dump(sessions, pickle_file)